In [13]:
import numpy as np
import pandas as pd
import collections
import pickle
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
import plotly.express as px
import plotly.graph_objects as go



In [2]:
#load in data
df2 = pickle.load( open( 'press_releases.pickle', "rb" ))
hist_members = pickle.load( open( 'hist_members.pickle', "rb" ))
curr_members = pickle.load( open( 'curr_members.pickle', "rb" ))

In [5]:
# gender = []
# for each in curr_members:
#     print(each)
#     gender.append(input())
# df_gender = pd.DataFrame(gender)
# df_gender.index = curr_members
# df_gender.reset_index(inplace=True)
# df_gender.columns=['name','gender']
# pickle.dump( df_gender, open( 'df_gender.pickle', "wb" ))
df_gender = pickle.load(open( 'df_gender.pickle', "rb" ))

In [6]:
contentArr =[]
dateArr = []
titleArr = []
#this is for every row
def destructureContent(row):
    content = row['content']
    contentSections_labeled = [ ]
    contentSections = content.split('\n') 
    for eachSection in contentSections:
        if not re.search('[.!:)"]+', eachSection[-5:-1]) and 1 < len(eachSection.split(' ')) < 20:
            contentSections_labeled.append((False, eachSection)) #find subtitles
        else: 
            contentSections_labeled.append((True, eachSection)) # find subsections
    newSection = '' 
    for eachSection in contentSections_labeled:
#         print(eachSection)
        
        if not eachSection[0] and len(newSection.split(' ')) > 8: #if false and the last previous section is longer than 8
            contentArr.append(newSection)
            titleArr.append(row.title)
            dateArr.append(row.date)
            newSection = eachSection[1]
            
        else:  # make more restrictive (20 words)
            newSection += " " + eachSection[1]
            
    contentArr.append(newSection)
    titleArr.append(row.title)
    dateArr.append(row.date)

df2.apply(destructureContent, axis=1)
df3 = pd.DataFrame({ 'date': dateArr, 'title': titleArr , 'content':contentArr,   })

In [7]:
def parse(df):
    alphanumeric = lambda x: re.sub(r'\w*\d\w*', ' ', x) # removes numbers
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) #removes punctuation
    no_nxt_line = lambda x: re.sub('#', ' ', x.lower()) #remove hash
    no_nxt_line = lambda x: re.sub('\n', ' ', x.lower()) #remove next lines

    df['content'] = df.content.map(alphanumeric).map(punc_lower).map(no_nxt_line)
    #convert date
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['term'] = df['year'].apply(lambda x: (int(x) - 2001) // 4)
    df['month'] = df['date'].dt.month
    df['yr_m'] =  df['year'].astype(str) + "_" + df['month'].astype(str)
    df = df[df['content'].str.len() > 30]
    return df
     
parse(df3)
parse(df2)

,date,title,content,year,term,month,yr_m
0,2016-06-03,"Speaker Melissa Mark-Viverito, Land Use Commit...",release council recommendations will mak...,2016,3,6,2016_6
2,2016-05-30,Statement by Speaker Melissa Mark-Viverito Re:...,“every year on memorial day we take time to ...,2016,3,5,2016_5
3,2016-05-03,Joint Statement by Council Speaker Melissa Mar...,re christopher park “the possibility that pr...,2016,3,5,2016_5
4,2016-05-03,"Speaker Mark-Viverito, Mayor de Blasio and Hea...",decrease followed stricter city laws multi a...,2016,3,5,2016_5
5,2016-05-03,New York City Council Speaker Melissa Mark-Viv...,bipartisan letter to speaker ryan and majorit...,2016,3,5,2016_5
...,...,...,...,...,...,...,...
1575,2020-01-23,Council Votes to Strengthen Lead Laws,the council will also vote to prohibit cashle...,2020,4,1,2020_1
1576,2020-01-27,Speaker Johnson and De Blasio Administration C...,starting today more eligible new yorkers can...,2020,4,1,2020_1
1577,2020-01-30,Speaker Corey Johnson Unveils Agenda to Addres...,long term comprehensive plan includes prevent...,2020,4,1,2020_1
1578,2020-01-31,Statement by Speaker Corey Johnson on the NYC ...,“the property tax commission has worked very ...,2020,4,1,2020_1


In [8]:
df3

,date,title,content,year,term,month,yr_m
0,2016-06-03,"Speaker Melissa Mark-Viverito, Land Use Commit...",release council recommendations will ma...,2016,3,6,2016_6
1,2016-05-30,Statement by Speaker Melissa Mark-Viverito Re:...,“every year on memorial day we take time to...,2016,3,5,2016_5
2,2016-05-03,Joint Statement by Council Speaker Melissa Mar...,re christopher park “the possibility that p...,2016,3,5,2016_5
3,2016-05-03,"Speaker Mark-Viverito, Mayor de Blasio and Hea...",decrease followed stricter city laws multi ...,2016,3,5,2016_5
4,2016-05-03,New York City Council Speaker Melissa Mark-Viv...,bipartisan letter to speaker ryan and majori...,2016,3,5,2016_5
...,...,...,...,...,...,...,...
1502,2020-01-23,Council Votes to Strengthen Lead Laws,the council will also vote to prohibit cashl...,2020,4,1,2020_1
1503,2020-01-27,Speaker Johnson and De Blasio Administration C...,starting today more eligible new yorkers ca...,2020,4,1,2020_1
1504,2020-01-30,Speaker Corey Johnson Unveils Agenda to Addres...,long term comprehensive plan includes preven...,2020,4,1,2020_1
1505,2020-01-31,Statement by Speaker Corey Johnson on the NYC ...,“the property tax commission has worked very...,2020,4,1,2020_1


In [14]:
# Create TF-IDF versions of the Count Vectorizers created earlier in the exercise
X = df3['content']    
tfidf_1 = TfidfVectorizer(ngram_range=(1,2), binary=True)
doc_word = tfidf_1.fit_transform(X)
words = tfidf_1.get_feature_names()

In [15]:
# def associateWithCouncil(curr_members):
dfArr = []
for eachmember in curr_members:
    if len(eachmember.split(' ')) > 2:
        eachmember = eachmember.replace(' Jr.', '')
        namesplit1 = eachmember.split(',')
    #         print(namesplit1[0])
        namesplit = namesplit1[0].split(' ') 
    #         print(namesplit[-1])
        last = namesplit[-1]
    else:
         last = eachmember
#     print(last.lower())
    if last.lower() == 'alicka ampry-samuel':
            last = 'ampry'
    index = words.index(last.lower()) # pull out index of word
    
    councilAssociations = doc_word[:,index].toarray() # pull out index of wrd in doc_word
#         print(councilAssociations)
#         print(np.sum(councilAssociations))
    dfArr.append(np.concatenate(councilAssociations))
df  = pd.DataFrame(dfArr)
df.index = curr_members
council_df = df.transpose()
council_df_2Viz = pd.DataFrame(council_df.apply(sum))
# council_df_2Viz.join(df_gender, on='index')

In [30]:
blue = '#4285f4'
red = '#db4437'
def showTopicsByCouncilPerson(council_df_2Viz):
    
    
    reset_council_df_2Viz = council_df_2Viz.reset_index()
#     print(reset_council_df_2Viz)
    
    reset_council_df_2Viz_merged = reset_council_df_2Viz.merge(df_gender, left_on='index', right_on='name')
    
    reset_council_df_2Viz_merged.columns = ['council seat', 'mentions','name', 'gender']
    
    reset_council_df_2Viz_merged.sort_values('mentions', inplace=True)
#     print(reset_council_df_2Viz_merged)
#     print(['found' if int(x) == 1  else 2 for x in np.array(reset_council_df_2Viz_merged['gender'])] )
    colors = [blue if int(x) == 1 else red for x in np.array(reset_council_df_2Viz_merged['gender'])]
    reset_council_df_2Viz_merged.to_json('councilseat_mentions.json')
    print(colors)
    
    fig = go.Figure()

    fig.add_trace(go.Bar({
        'y': reset_council_df_2Viz_merged['council seat'],
        'x': reset_council_df_2Viz_merged['mentions'],
        'name':'coincil member mentions',
        'orientation':'h',
#         'color':reset_council_df_2Viz_merged['mentions'],
        'marker':dict(
            color=colors
#             line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    }))
    
#     fig = px.bar(reset_council_df_2Viz_merged, x="mentions", y="council seat", orientation='h', color_discrete_sequence=colors)
    fig.show()
    
showTopicsByCouncilPerson(council_df_2Viz)

['#4285f4', '#db4437', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#db4437', '#4285f4', '#db4437', '#4285f4', '#4285f4', '#db4437', '#4285f4', '#db4437', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#db4437', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#db4437', '#4285f4', '#4285f4', '#db4437', '#4285f4', '#db4437', '#4285f4', '#db4437', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#4285f4', '#db4437', '#4285f4', '#4285f4', '#4285f4']


In [20]:
# build doc word after features and city council 
features_less3 = [f for f in tfidf_1.get_feature_names() if len(f) <= 3]

def parseStopWords(legal_text):
    legal_text = re.sub(r'\w*\d\w*', ' ', legal_text) # removes numbers
    legal_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', legal_text.lower()) #removes punctuation
    legal_text = re.sub('\n', ' ', legal_text.lower()) #remove next linesre.sub('\n', ' ', legal_text.lower()) #remove next lines
    return list(set(legal_text.split(' ')))

def parseMembers(arr):
    newArr = []
    for eachMember in arr:
        listOfNames = eachMember.split(" ")
        for eachName in listOfNames:
            newArr.append(eachName.lower())
    return newArr
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday' , 'saturday' , 'sunday']
months = ['january', 'jan', 'winter','february', 'feb','march', 'mar', 'spring','april', 'apr' , 'may', 'may'
,'june', 'jun', 'summer','july', 'jul','august', 'aug','september', 'sep', 'autumn'
, 'october', 'oct', 'november', 'nov', 'december', 'dec', 'winter']
legal_stopwords = parseStopWords(legal_text)
NYC_geography_stopwords = parseStopWords(NYC_geography)
hist_members_stopwords = parseMembers(hist_members)
curr_members_stopwords = parseMembers(curr_members)


X1 = df2['content']
X2 = df3['content'] 

stop_words = text.ENGLISH_STOP_WORDS.union(legal_stopwords, features_less3, days, months,  hist_members_stopwords, curr_members_stopwords, NYC_geography_stopwords)

tfidf = TfidfVectorizer(ngram_range=(1,2), binary=True, stop_words=stop_words)
doc_word_X1 = tfidf.fit_transform(X1)
doc_word_X2 = tfidf.transform(X2)

words = tfidf.get_feature_names()


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:385: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1658', '1742', 'ampry', 'businessman', 'deputy', 'diaz', 'dwyer', 'eristoff', 'judge', 'politician', 'sidamon'] not in stop_words.



In [21]:
from sklearn.decomposition import NMF
nmf_model = NMF(50, random_state=1)
doc_topic = nmf_model.fit_transform(doc_word_X1)

# # nmf_model = pickle.load(open('nmf_model.pickle', 'rb'))
# # doc_topic = pickle.dump(open('doc_topic.pickle', 'rb'))

# pickle.dump(nmf_model, open('nmf_model.pickle', 'wb'))
# pickle.dump(doc_topic, open('doc_topic.pickle', 'wb'))


In [24]:

# topics = ['general','general', 'general', 'hate crimes', 'building requirements', 
#           'general','the environment','civil rights', 'tenants rights' , 'trajedy','social services',
#           'business permitting','general','unemployment','water safety', 'reproductive health', 
#           'disaster','urban planning','commuters', 'general','general','self defense','condolences', 
#           'policing', 'child development', 'property taxes','condolences', 'gay marriage', 'hiv', 
#           'housing affordability','food','civic engagement','general','child internet safety',
#           'reproductive health','gang violence', 'blood donations','veterans','housing affordability','affordable housing','general','general',
#           'fire','the environment','schools','voting','homelessness','tourism','energy','general']
#self defense food stamps and healthy food child internet safety social services 'veterans',
def findTopics(): 
    words = tfidf.get_feature_names()
    components = nmf_model.components_
    print(len(words), components.shape)
    topic_words = []
    topicLookup = []
    i = 0
    for r in components:
        a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:30] # for each row, 
        i+=1
        print(i)
#         print(topics[i-1])
        print([words[e[1]] for e in a])
#         topicLookup.append((topics[i-1],[words[e[1]] for e in a]))
        topic_words.append([words[e[1]] for e in a]) # word_ids are indexed from 1, not 0
    return topic_words 

def matchWord(eachSetofnewWords, eachPair):
    topic = eachPair[0]
    oldWords = eachPair[1]
    i=0
    for eachWord in eachSetofnewWords:
        if eachWord in oldWords:
#             print(i,eachWord)
            i+=1
        if i > 6:
#             print(topic, eachSetofnewWords)
            if topic == 'the environment':
                print(eachSetofnewWords)
                topic = input()
            return (topic,eachSetofnewWords)
    #no match was found
    return False
    
def matchTopics(topic_words,topicLookup):
    newTopicLookup = []  

    for eachSetofnewWords in topic_words: #new set []
        logFlag = True
        for eachPair in topicLookup: #pair in old lookup (tuple, )
            if matchWord(eachSetofnewWords, eachPair):
                newTopicLookup.append(matchWord(eachSetofnewWords, eachPair))
                logFlag = False
                break
        if logFlag: 
            print(eachSetofnewWords)
            unknowntopic = input()
            newTopicLookup.append((unknowntopic, eachSetofnewWords))
            
    return newTopicLookup

#save pickle of dict

topic_words = findTopics()
# pickle.dump(matched_topics, open('topicLookup.pickle', 'wb'))
topicLookup = pickle.load(open('topicLookup.pickle', 'rb'))
# print(topicLookup)
matched_topics = matchTopics(topic_words, topicLookup)
# print(len(matched_topics))

311604 (50, 311604)
1
['funding', 'julissa ferreras', 'ferreras', 'julissa', 'investment', 'million funding', 'youth employment', 'youth', 'employment program', 'million', 'investing', 'commitment', 'senior', 'ferreras copeland', 'copeland', 'cultural affairs', 'resources', 'cultural', 'libraries', 'priorities', 'employment', 'aging', 'investments', 'funding support', 'blasio', 'care', 'seniors', 'vital', 'fund', 'organizations']
2
['hate', 'hate crimes', 'crimes', 'nypd hate', 'crimes task', 'crime', 'stoppers', 'crime stoppers', 'nypd', 'stoppers hotline', 'thank nypd', 'hotline', 'task', 'hateful', 'urge', 'hate crime', 'contact crime', 'acts', 'violence', 'anti', 'tolerated', 'slurs', 'contact', 'urge contact', 'attack', 'incident', 'swift', 'diversity', 'cowardly', 'attacks']
3
['require', 'requiring', 'sponsored', 'sponsored require', 'require department', 'requires', 'finally', 'reporting', 'package', 'regarding', 'buildings', 'department', 'additionally', 'requirements', 'post'

19
['years inspired', 'years ideas', 'wrote waged', 'writer activist', 'working tenant', 'work jane', 'work doris', 'work death', 'waged successful', 'waged', 'vision planning', 'urban theorist', 'urban landscapes', 'urban activists', 'urban activism', 'unveiling jane', 'universally supported', 'understood human', 'turning points', 'truth hear', 'triumph destructive', 'tributes history', 'transformational work', 'touch professionals', 'today heretical', 'tierney pleased', 'tierney community', 'theorist articulated', 'theorist', 'theories relevant']
20
['property fair', 'value percent', 'validate methodology', 'undermined confidence', 'transparent confines', 'today road', 'today admitting', 'taxpayers fair', 'taxpayers failed', 'taxpayers counts', 'taxpayers administration', 'taxable value', 'suffered increases', 'structured review', 'source today', 'source significant', 'shortcomings assessments', 'share confident', 'road restoring', 'review validate', 'resulting assessments', 'restori

35
['fair fares', 'strengthening social', 'subcommittee capital', 'reserves million', 'capital subcommittee', 'latonia mckinney', 'latonia', 'director latonia', 'million reserves', 'student funding', 'fair student', 'social safety', 'resources yorkers', 'mckinney', 'fares', 'reform property', 'reform commission', 'metrocards', 'weather downturn', 'truly delivers', 'pushed improve', 'progressive truly', 'priorities pushed', 'priorities increased', 'picking trash', 'includes priorities', 'extra sanitation', 'subcommittee director', 'mckinney team', 'blasio capital']
36
['vacation travel', 'typically donations', 'travel donations', 'supplies typically', 'staffers hand', 'schools donate', 'regional blood', 'products patients', 'patients need', 'participate bringing', 'hosted blood', 'hand effort', 'effort blood', 'donations tend', 'donations staffers', 'donations schools', 'donate wish', 'colleges account', 'blood products', 'blood drive', 'blood collection', 'account regional', 'blood don

environment
['yogurt tubs', 'tubs containers', 'tubs', 'sponsored household', 'rigid plastic', 'rigid', 'recycling goals', 'recycling coordinator', 'receptacles classroom', 'plastics', 'plastic containers', 'medicine bottles', 'leaf yard', 'leaf', 'items yogurt', 'increasing events', 'household hazardous', 'hazardous waste', 'expand recycling', 'events making', 'coordinator recycling', 'containers including', 'commercial recycling', 'collection event', 'classroom locations', 'bins spaces', 'waste composting', 'recycling receptacles', 'walking expanding', 'recycle things']
environment


In [25]:
matched_topics

[('community development',
  ['funding',
   'julissa ferreras',
   'ferreras',
   'julissa',
   'investment',
   'million funding',
   'youth employment',
   'youth',
   'employment program',
   'million',
   'investing',
   'commitment',
   'senior',
   'ferreras copeland',
   'copeland',
   'cultural affairs',
   'resources',
   'cultural',
   'libraries',
   'priorities',
   'employment',
   'aging',
   'investments',
   'funding support',
   'blasio',
   'care',
   'seniors',
   'vital',
   'fund',
   'organizations']),
 ('hate crimes',
  ['hate',
   'hate crimes',
   'crimes',
   'nypd hate',
   'crimes task',
   'crime',
   'stoppers',
   'crime stoppers',
   'nypd',
   'stoppers hotline',
   'thank nypd',
   'hotline',
   'task',
   'hateful',
   'urge',
   'hate crime',
   'contact crime',
   'acts',
   'violence',
   'anti',
   'tolerated',
   'slurs',
   'contact',
   'urge contact',
   'attack',
   'incident',
   'swift',
   'diversity',
   'cowardly',
   'attacks']),
 ('bui

In [60]:
topics = [x[0] for x in topics_filtered if x[0] == 'budget_']
len(topics)

0

In [26]:
topics = [x[0] for x in matched_topics]
doc_topic_fitted = nmf_model.transform(doc_word_X2)
doc_topic_df = pd.DataFrame(doc_topic_fitted)
doc_topic_df.columns = topics
doc_topic_df.drop(columns=['general'], inplace=True)

for each in [item for item, count in collections.Counter(doc_topic_df.columns).items() if count > 1]:
    doc_topic_df[each+'_'] = doc_topic_df[each].iloc[:,0] + doc_topic_df[each].iloc[:,1]

doc_topic_df.drop(columns=[item for item, count in collections.Counter(doc_topic_df.columns).items() if count > 1], inplace=True)
topics_filtered = doc_topic_df.columns
df4 = pd.concat([df3, doc_topic_df, council_df], axis=1) # 

In [28]:
df4

,date,title,content,year,term,month,yr_m,community development,hate crimes,building requirements,...,Inez Barron,Justin Brannan,Kalman Yeger,Farah N. Louis,Alan N. Maisel,Mark Treyger,Chaim M. Deutsch,Deborah Rose,Steven Matteo,Joseph C. Borelli
0,2016-06-03,"Speaker Melissa Mark-Viverito, Land Use Commit...",release council recommendations will ma...,2016,3,6,2016_6,0.001109,0.000000,0.037961,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,2016-05-30,Statement by Speaker Melissa Mark-Viverito Re:...,“every year on memorial day we take time to...,2016,3,5,2016_5,0.000344,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,2016-05-03,Joint Statement by Council Speaker Melissa Mar...,re christopher park “the possibility that p...,2016,3,5,2016_5,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,2016-05-03,"Speaker Mark-Viverito, Mayor de Blasio and Hea...",decrease followed stricter city laws multi ...,2016,3,5,2016_5,0.001907,0.001934,0.030041,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,2016-05-03,New York City Council Speaker Melissa Mark-Viv...,bipartisan letter to speaker ryan and majori...,2016,3,5,2016_5,0.000549,0.000000,0.003103,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,2020-01-23,Council Votes to Strengthen Lead Laws,the council will also vote to prohibit cashl...,2020,4,1,2020_1,0.000000,0.000000,0.082962,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.018111,0.0,0.0,0.0
1503,2020-01-27,Speaker Johnson and De Blasio Administration C...,starting today more eligible new yorkers ca...,2020,4,1,2020_1,0.004282,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1504,2020-01-30,Speaker Corey Johnson Unveils Agenda to Addres...,long term comprehensive plan includes preven...,2020,4,1,2020_1,0.018839,0.000000,0.015493,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1505,2020-01-31,Statement by Speaker Corey Johnson on the NYC ...,“the property tax commission has worked very...,2020,4,1,2020_1,0.000000,0.000220,0.006140,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [27]:

def buildTimeSeries(df):
    #group by month
    df_byMonth = df.groupby(['yr_m','year', 'term']).agg('sum').reset_index()
    df_byMonth['dt'] = pd.to_datetime(df_byMonth['yr_m'].apply(lambda x: x[:4]+'/'+x[5:]+  '/01') )
    
    fig = go.Figure()
    for i in range(35):
        fig.add_trace(go.Bar(name=topics_filtered[i], x=df_byMonth['dt'], y=df_byMonth[topics_filtered[i]]))  
    fig.add_trace(go.Scatter(name='election cycle' ,line=dict(color="black"), x=df_byMonth['dt'], y=[-(x%2)*2-2 for x in df_byMonth['term']]))

    fig.update_layout(barmode='stack')
    fig.show()   

buildTimeSeries(df4)

In [59]:
topics_filtered

Index(['community development', 'hate crimes', 'building requirements',
       'the environment', 'tenants rights', 'business permitting', 'food',
       'self defense', 'voting', 'social services', 'disaster',
       'reproductive health', 'urban planning', 'property taxes', 'commuters',
       'child development', 'civic engagement', 'schools', 'hiv',
       'early childhood education', 'transportation', 'child internet safety',
       'veterans', 'gang violence', 'policing', 'blood donations', 'min wage',
       'fire', 'homelessness', 'unemployment', 'public health',
       'gay marriage_', 'trajedy_', 'budget_', 'employment_', 'condolences_',
       'housing affordability_', 'immigration_', 'affordable housing_',
       'environment_'],
      dtype='object')

In [32]:
# build bars
from plotly.offline import init_notebook_mode, iplot
   
def findAverage():
    dictToDF = {}
    for each in topics_filtered:
        dictToDF[each] = [df4[each].sum()/50]
#     print(dictToDF)
    TopicsDF = pd.DataFrame.from_dict(dictToDF)
    TopicsDF_transposed = TopicsDF.transpose().reset_index() 
    TopicsDF_transposed.columns = ['issue', 'avg_level']
#     TopicsDF_transposed.sort_values('level', inplace=True)
#     fig = px.bar(TopicsDF_transposed, x="level", y="issue", orientation='h')
#     fig.show()    
#     buildTimeSeries(df_filtered)

#     print(TopicsDF_transposed)
    return TopicsDF_transposed
    
    

def showTopicsByCouncilPerson(name, TopicsDF_AVG):
    df_filtered = df4[name] > 0
#     print(df_filtered.shape)
    dictToDF = {}
    for each in topics_filtered:
        dictToDF[each] = [df_filtered[each].sum()]
    TopicsDF = pd.DataFrame.from_dict(dictToDF)
    TopicsDF_transposed = TopicsDF.transpose().reset_index()    
    
    TopicsDF_transposed.columns = ['issue', 'level']
#     joint_DF = TopicsDF_transposed.merge(TopicsDF_AVG, on='issue' )
#     print(joint_DF)
    joint_DF = TopicsDF_transposed

    joint_DF.sort_values('level', ascending=False, inplace=True)
#     print(joint_DF)
    fig = {
    'data': [go.Bar(y=joint_DF['level'],  x=joint_DF['issue'], name='council person')],
#              go.Bar(x=joint_DF['avg_level'], y=joint_DF['issue'], orientation='h',width=[.2 for i in range(TopicsDF_transposed.shape[0])], name='council avg')], #width=[.5,]
    'layout': go.Layout(barmode='overlay', xaxis_tickangle=-90, xaxis=dict(
        tickfont_size=14,
    ))
    } 
    
    
    
    iplot(fig)
    
#     fig = go.Figure()  
#     fig.add_trace(px.bar(TopicsDF_transposed, x="level", y="issue", orientation='h'))

#     fig.update_layout(barmode='stack')
#     fig.show()  
    
#     fi
#     fig = 
#     fig.show()
    
    buildTimeSeries(df_filtered)
    
TopicsDF_AVG = findAverage()
showTopicsByCouncilPerson('Farah N. Louis', TopicsDF_AVG)
# showTopicsByCouncilPerson('Peter Koo',TopicsDF_AVG)




KeyError: 'community development'

In [70]:
# curr_members

def buildComparison(df4):
    threeDimDict = {}
    
    threeDimDict['CPeople'] = curr_members
    
    topicsToCompare = ['hate crimes', 'building requirements', 'the environment','community development']
    
    for eachTopic in topicsToCompare:
        threeDimDict[eachTopic] = []
    
    def pullDimsByPerson(name):
        df_filtered = df4[df4[name] > 0]
        for eachTopic in topicsToCompare:            
            threeDimDict[eachTopic].append(df_filtered[eachTopic].sum())

        
    for eachPerson in curr_members:
        pullDimsByPerson(eachPerson)
        
    return pd.DataFrame(threeDimDict)

threeDimDictDF = buildComparison(df4)   

In [202]:
fig = px.scatter_3d(threeDimDictDF, x='hate crimes', y='building requirements', z='the environment',
              color='CPeople', opacity=0.7)

fig.show()

In [16]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=stop_words,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=1000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(df3.content)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1658', '1742', 'ampry', 'businessman', 'deputy', 'diaz', 'dwyer', 'eristoff', 'guez', 'hell', 'jos', 'judge', 'pag', 'politician', 'rodr', 'sidamon', 'vel', 'zquez'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [17]:
# Build LDA Model
n_components = 5
lda_model = LatentDirichletAllocation(n_components=n_components,   # Number of topics
                                      max_iter=5,                 # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,            # Random state
                                      #batch_size=128,             # n docs in each learning iter
                                      n_jobs = -1,                 # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=5, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [18]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  21.170192
2       51.163364 -159.968018       2        1  20.986731
1      -73.144554   21.523815       3        1  19.858910
3      -66.410133 -110.606438       4        1  19.144691
4       54.417324   12.443444       5        1  18.839475, topic_info=    Category         Freq       Term        Total  loglift  logprob
431  Default  2382.000000    housing  2382.000000  30.0000  30.0000
797  Default  1642.000000     school  1642.000000  29.0000  29.0000
367  Default  1418.000000       food  1418.000000  28.0000  28.0000
279  Default  1517.000000  education  1517.000000  27.0000  27.0000
166  Default  3210.000000  community  3210.000000  26.0000  26.0000
..       ...          ...        ...          ...      ...      ...
986   Topic5   380.234716       work  2092.175137  -0.0360  -4.8402
583   Topic5   337.079189       need  1567.744728   0.1321  -4.9607
166   Topic5   334.907356  community  3210.528617  -0.5911  -4.9671
699   Topic5   271.719837    protect   670.178990   0.7664  -5.1762
505   Topic5   276.818925    justice  1022.468522   0.3626  -5.1576

[325 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         2  0.994647  abatement
7         3  0.990338       acts
11        1  0.085807    address
11        2  0.191975    address
11        3  0.389767    address
...     ...       ...        ...
998       4  0.125027      youth
998       5  0.680547      youth
999       2  0.072373     zoning
999       3  0.839528     zoning
999       4  0.086848     zoning

[698 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5])

In [18]:

legal_text = """ ABSENT: Not present at a session.
council 
mayor
said
project 
cornegy
finance


Absent with leave: Not present at a session with consent.
Absent without leave: Not present at a session without consent.
ACT: Legislation enacted into law. A bill that has passed both houses of the legislature, been enrolled, ratified, signed by the governor or passed over the governor's office, and printed. It is a permanent measure, having the force of law until repealed.
Local act: Legislation enacted into law that has limited application.
Private act: Legislation enacted into law that has limited application.
Public act: Legislation enacted into law that applies to the public at large.
ADHERE: A step in parliamentary procedure whereby one house of the legislature votes to stand by its previous action in response to some conflicting action by the other chamber.
ADJOURNMENT: Termination of a session for that day, with the hour and day of the next meeting being set.
ADJOURNMENT SINE DIE: Final termination of a regular or special legislative session.
ADOPTION: Approval or acceptance; usually applied to amendments, committee reports or resolutions.
AMENDMENT: Any alteration made (or proposed to be made) to a bill or clause thereof, by adding, deleting, substituting or omitting.
Committee amendment: An alteration made (or proposed to be made) to a bill that is offered by a legislative committee.
Floor amendment: An alternation offered to a legislative document that is presented by a legislator while that document is being discussed on the floor of that legislator's chamber.
APPEAL: A parliamentary procedure for testing (and possibly changing) the decision of a presiding officer.
APPORTIONMENT: Establishment of the legislative districts from which members are elected.
APPROPRIATION: Funds allocated for various departments of government set aside by formal action for specific use. Allows money to be spent; is not actual expenditure record.
AT-LARGE ELECTION: An election in which candidates are chosen on an individual basis rather than as representatives of a geographically defined, single-member district. At-large elections can be held at the legislative and presidential levels. In the United State of America, some states hold at-large elections for congressional seats, when, for instance, a state's entire population warrants only one representative.
AUTHOR: The person (usually a legislator) who presents a bill or resolution for consideration; may be joined by others, who are known as coauthors. See also: introducer, patron, sponsor.
BICAMERAL: A legislature consisting of two separate chambers, each serving as a check on the other's power.
BIENNIUM: Two-year term of legislative activity.
BILL: Draft of a proposed law presented to the legislature for consideration.
BIPARTISAN: Having an affiliation or association with (or representatives of) both political parties or caucuses in a twotwo-partystem.
BUDGET: (1) The suggested allocation of state moneys presented to the legislature for consideration; (2) a formal document that reflects the authorized expenditures of the state.
CALENDAR: (1) A printed list of proposals that are arranged according to the order of business and are scheduled for consideration by a chamber. (2) Agenda of daily legislative business in a chamber.
CALENDAR DAY: Literally a day as listed on the Gregorian calendar.
CALL OF THE SENATE OR HOUSE: Procedure used to compel the attendance of members who are missing from the chamber and to compel those members already in attendance to remain in the chamber.
CARRY-OVER LEGISLATION: Legislation that is held over from the first year of a legislative biennium to the second year.
CAUCUS: An informal meeting of a group of the members; most commonly based on political party affiliation, but may have other bases, such as gender, race, geographic location or specific issue.
CENSURE: An action by a legislative body to officially reprimand an elected official for inappropriate or illegal actions committed by that official while in office. The act of censuring is an official condemnation for inappropriate or illegal actions committed by a public official while holding a position of trust.
CHAMBER: Official hall for the meeting of a legislative body.
CLERK OF THE HOUSE OR ASSEMBLY: A non-legislator officer who is appointed or elected by the members of the House of Representatives or Assembly to perform and direct the parliamentary and clerical functions of the chamber. Also may be titled "chief clerk" or "principal clerk."
CODE: A compilation of laws and their revisions according to subject matter (usually arranged by title, chapter and section); the official publication of the statutes.

COMMITTEE: A body of members appointed by the presiding officer (or another authority specified by the chamber) to consider and make recommendations concerning disposition of bills, resolutions and other related matters.

Conference committee: A committee composed of members from the two houses specifically appointed to reconcile the differences between House and Senate versions of a bill or bills.
Interim committee: A committee established to study or investigate certain matters between annual or biennial legislative sessions and to report to the next regular session.
Joint committee: A committee composed of members from both chambers.
Standing committee: A committee appointed with continuing responsibility in a general issue area or field of legislative activity.
COMMITTEE OF THE WHOLE: Either house of the legislature sitting in its entirety as a committee to consider bills or issues.

COMMITTEE REPORT: Official release of a bill or resolution from committee with (or without) a specific recommendation, such as "pass", "pass as amended" or "do not pass."

COMMITTEE SUBSTITUTE: A bill offered by a committee in lieu of another bill that was originally referred to the committee for consideration; technically, the committee substitute is an amendment to the original bill.

CONCURRENCE (TO CONCUR): Action by which one house agrees to a proposal or action that the other chamber has approved.

CONFLICT OF INTEREST: Untenable position that threatens the ability of a legislator to vote impartially due to some personal interest in a legislative issue.

CONSTITUENT: A citizen residing within the district of a legislator.

CONSTITUTION: A written instrument embodying the fundamental principles of the state that guarantees and duties of the government and guarantees certain rights to the people.

CONSTITUTIONAL MAJORITY: One more than half of the members of a deliberative body; actual number may be defined in the state constitution.

CONVENE: When the members of a chamber gather for the meeting of the legislature daily, weekly and at the beginning of a session as provide by the constitution or law.

DEBATABLE: Open to parliamentary discussion or argument.

DEBATE: Discussion of a matter according to parliamentary rules.

DECORUM: Proper order, etiquette and conduct of members during a floor session.

DIED IN COMMITTEE: The defeat of a bill by not returning it from committee to the house for further action. (Only permitted in certain states.)

DILATORY: Deliberate use of parliamentary procedure to delay.

DISSENT: Difference of opinion; to cast a negative vote.

DISTRICT: That division of the state represented by a legislator distinguished numerically or by geographical boundaries.

DIVISION: A method of voting; a request that members stand or raise hands to be counted when the outcome of a voice vote is unclear or in dispute.

DIVISION OF A QUESTION: Procedure to separate a matter to be voted upon into two or more questions.

EFFECTIVE DATE: A law generally becomes effective, or binding, either upon a date specified in the law itself or, in the absence of such a date, a fixed number of days (depending on the state) after the final adjournment of the session during which it was enacted or on signature by the governor.

ELECTION: Act of selecting a person to fill an office.

EMERGENCY CLAUSE: A statement in a bill that indicates the act shall take immediate effect.

ENACTING CLAUSE: That clause of an act that formally expresses the legislative sanction. It varies in different states, but usually begins "Be It Enacted."

ENGROSS: Most commonly, the process by which a bill is updated--that is, how adopted amendments and other changes are incorporated into a bill—as it makes its way through the Senate or House.

ENROLL: Most commonly, the process of changing a bill passed by both chambers into its final format for transmission to the governor.

EXECUTIVE SESSION: A session excluding from the chamber or committee room all persons other than members and essential staff personnel.

EXCUSED: Absent with the permission of the body or the presiding officer.

EXPUNGE: An action that directs the removal of specific portions from the journal. This is applicable in situations where objectionable, inflammatory or incorrect matter has been included in the journal.

FILIBUSTER: The prolonged discussion of a bill to delay legislative action.

FIRST READING: The first presentation of a bill or its title for consideration. In some states, the first reading is done at the time of introduction.

FISCAL: Dealing with state revenues and expenditures.

FISCAL NOTE: A fiscal note seeks to state in dollars the estimated amount of increase or decrease in revenue or expenditures and the present and future implications of a piece of pending legislation.

FISCAL YEAR: An accounting period of 12 months.

FLOOR: That portion of the legislative chamber reserved for members and officers of the assembly or other persons granted privileged access.

GALLERY: Balconies of the chamber from which visitors may view the proceedings of the legislature.

GERMANENESS: The relevance or appropriateness of amendments or substitutes.

HEARING: Public discussion and appearance on a proposal or bill; usually scheduled by a committee.

HOUSE: Generic term for a legislative body; usually the body in a bicameral legislature that has the greater number of members; shortened name for House of Representatives or House of Delegates.

IMPEACHMENT: Procedure to remove from office a public official accused of misconduct.

INDEFINITE POSTPONEMENT: A form of adverse disposition of a proposal for that session of the legislature.
INSERT: Add language to a bill or resolution.
INTERIM: The interval between regular sessions of the legislature.
INTRODUCER: The person (usually a legislator) who presents a bill or resolution for consideration; may be joined by others, who are known as cointroducer. See also: author, patron, sponsor
INTRODUCTION: The formal presentation of a proposal after it has been drafted.
ITEM VETO: An action taken by the governor to prevent the enactment of an item of an appropriation bill; also may be called line item veto.
JOINT RULES: Parliamentary rules governing joint procedures or operations of the Senate and House.
JOINT SESSION: A combined meeting of the Senate and House in one chamber.
JOURNAL: An official chronological record of the actions taken and proceedings of the respective chambers.
LEGISLATIVE DAY: A day on which either chamber convenes (or both chambers convene) to conduct official business.
LEGISLATIVE INTENT: Purpose for which a measure is passed.
LEGISLATIVE LIAISON: Person appointed to communicate between legislators and other departments.
LEGISLATIVE OVERSIGHT: Scrutiny of executive branch programs and performance by the legislature.
LEGISLATIVE SERVICE AGENCY: Nonpartisan legislative branch agency providing services such as legal and bill drafting, impartial research and information or technical services.
LEGISLATOR: Elected member of a legislative body.
LEGISLATURE: The branch of state government responsible for enacting laws.
LINE ITEM: Numeric line in an appropriation or budget bill.
LOBBYIST: A representative of a special interest group whose function is to influence legislation affecting his special interest.
MAJORITY LEADER: A member of the majority political party designated to be a leader. (The procedure for designating the majority leader and other officers varies from state to state.)
MAJORITY PARTY: The political party having the greatest number of members in the legislature or in either chamber.
MAJORITY REPORT: Recommendation of action on a measure that is submitted by a majority of the members of a committee.
MEASURE: General term for bill, resolution or memorial.
MEMBER ELECT: Member who has been elected, but who has not yet taken the oath of office or who is not yet officially serving.
MEMBERS PRESENT: The term used to refer to those members who are actually present at a daily session.
MEMORIAL: The method by which the legislature addresses or petitions Congress and other governments or governmental agencies; method by which the legislature congratulates or honors groups or individuals.
MINORITY LEADER: A member of the minority political party designated to be leader. (Process of designation varies from state to state.)
MINORITY PARTY: The political party having fewer numbers of members in the legislature or in either chamber.
MINORITY REPORT: A report that reflects the thinking of the members not favoring the majority position or action on an issue.
MINUTES: Accurate record of the proceedings of a meeting in chronological order.
MOTION: Formal proposal offered by a member of a deliberative assembly.
NONPARTISAN: Having no association or affiliation with a political party or caucus.
OATH OF OFFICE: Oath taken by members-elect of the legislature prior to being seated and embarking upon official duties.
ORDER OF BUSINESS: The defined routine of procedure in the legislative body each day.
OUT OF ORDER: Not being conducted under proper parliamentary rules and procedures.
PARLIAMENTARY INQUIRY: Question posed by a member to the presiding officer for clarification of the procedure or business before the house.
PARTISAN:- Associated or affiliated with a single political party or caucus.
PATRON: The person (usually a legislator) who presents a bill or resolution for consideration; may be joined by others, who are known as copatrons. See also: author, introducer, sponsor
PER DIEM: Literally, per day; daily expense money rendered to legislators or staff.
PETITION: Formal request submitted by an individual or group of individuals to the legislature.
POINT OF INFORMATION: A request from a legislator to the presiding officer for clarification of a procedural matter.
POINT OF ORDER: A question by a member to the presiding officer calling attention to a breach of order or of the rules.
POSTPONE INDEFINITELY: A means of disposing of an issue by not setting a date on which to consider it again.
PRECEDENT: Interpretation of rulings by presiding officers on specific rules; unwritten rules that are established by custom.
PREFILE: Ability to introduce a measure before the opening of the session.
PRESIDENT: Usually, the title given to the person elected (or designated by constitution) as the presiding officer of the Senate.
PRESIDING OFFICER: Person designated to preside at a legislative session.
PREVIOUS QUESTION: A motion to close debate and bring the pending question or questions to an immediate vote.
PRO TEMPORE (PRO TEM): The designated officer of the senate or house acting in the absence of the regular presiding officer.
QUORUM: When a legislative body is assembled, the minimum number of members required to transact business.
QUORUM CALL: A method used to establish the presence of a majority for the lawful transacting of business.
RATIFY: To approve and make valid.
READING: Presentation of a bill before either chamber by the reading the bill, its title or its number. A formal procedure required by constitution and rules that indicates a stage in enactment process. Most often, a bill must receive three readings on three different days in each legislative body.
REAPPORTIONMENT: Redrawing legislative district boundaries to provide equality of representation.
RECESS: Intermission in a daily session; intermission from one day to the next; .
REFERRAL: The assigning or referring of a bill to committee.
REPEAL: A method by which a legislative action is revoked or annulled.
RESOLUTION: A document that expresses the sentiment or intent of the legislature or a chamber, that governs the business of the legislature or a chamber, or that expresses recognition by the legislature or a chamber.
ROLL CALL: Names of the members being called in alphabetical order and recorded; used to establish a quorum or to take a vote on an issue before the body.
RULES: Regulating principles or methods of legislative procedure.
RULING OF THE CHAIR: A decision by the presiding officer concerning a question of order or procedure.
SECRETARY OF THE SENATE: A non-legislator officer appointed or elected by the members of the Senate to perform and direct the parliamentary and clerical functions of the Senate; also may be called "clerk," "chief clerk" or "principal secretary."
SENATE: A legislative body; usually the body in a bicameral legislature having the fewer number of members.
SENIORITY: Recognition of prior legislative service.
SESSION: (1) Period during which the legislature meets; (2) the daily meeting of the Senate or House.
Regular session: The annual (or biennial) meeting of the legislature required by constitution
Special (or extraordinary) session: A special meeting of the legislature that is called by the governor (or the legislature itself) and limited to specific matters.
SIMPLE MAJORITY: One more than half of those voting on a question.
SINE DIE: Literally, 'without day;' usually, adjournment without a day being set for reconvening; final adjournment.
SPEAKER: Usually the title given to the person elected as the presiding officer of the House or Assembly; in some states, the title given to the presiding officer of the Senate.
SPECIAL ORDER: (1) To set consideration of a bill or measure for a specific, future time of the session; (2) matter of business set for discussion at a special time, on a designated day or both.
SPONSOR: The person (usually a legislator) who presents a bill or resolution for consideration; may be joined by others, who are known as cosponsor. See also: author, introducer, patron.
STATE: One of the 50 units constituting the United States of America.
STATUTE: A formal enactment of the legislature of a more permanent nature. The term "statute" is used to designate written law, as distinguished from unwritten law.
STATUS OF BILL: The progress of a bill at any given time in the legislative process. It can be in committee, on the calendar, in the other house, etc.
STRIKE OUT: The deletion of language from a bill or resolution.
SUNSET: Expiration date of a measure.
SUPPLEMENTAL APPROPRIATION: Adjustment of funds allocated by the original appropriation.
SUSPENSION OF THE RULES: Parliamentary procedure whereby actions can be taken that would otherwise be out of order.
TERM OF OFFICE: Period of time for which a person is elected.
TITLE: A concise statement of the subject and the contents of a bill.
UNICAMERAL: A legislature with only one chamber.
VETO: Action by the governor to disapprove a measure.
VETO OVERRIDE: Vote by the legislature to pass a bill over a governor's veto.
VOICE VOTE: Oral expression of the members when a question is submitted for their determination. When asked by the presiding officers, members respond "aye" or "nay." The presiding officer then decides which side prevailed.
VOTE: Formal expression of a decision by the body.
YEAS AND NAYS: Recorded vote of members on an issue.
YIELD: To relinquish the floor to another member to speak or ask question."""


In [19]:
NYC_geography = """ 

The Five Boroughs
Manhattan: Manhattan is the area most people think of when they think of New York City. It’s the home of the Empire State Building, Rockefeller Center, the 9/11 Museum & Memorial, Central Park, and Times Square. Manhattan is its own island, a long and slender landmass 13.4 miles long and 2.3 miles wide at its widest point. The other four boroughs encircle Manhattan, and New Jersey lies across the Hudson River on its western side. The island is easily accessible for watercraft to access via the Atlantic Ocean.


Brooklyn: Brooklyn is separated from Manhattan by the East River and sits at the western tip of Long Island. Brooklyn’s wide beachfront on the south end is where you’ll find the famous Luna Park at Coney Island overlooking the Atlantic Ocean. On the north side (below Queens) are the neighborhoods Greenpoint and Williamsburg. Nearly 70 square miles in size, Brooklyn is connected to Manhattan via three , the most famous which is the Brooklyn Bridge, a 1.1 mile distance for millions of cars, bicycles, and pedestrians each day.


 
Queens: Queens lies above Brooklyn, occupying the north side of Long Island’s western tip. Queens is situated between the Bronx and Manhattan, which both lie on the other side of the East River. Primarily a residential area, Queens is where you’ll find both of New York City’s major airports at LaGuardia and JFK. The Queens neighborhood closest to the East River is Long Island City (offering gorgeous views of the Manhattan skyline), while its easternmost neighborhoods include Cambria Heights and Floral Park. Beyond the eastern boundary of Queens is the remainder of Long Island, where you’ll find residential  and beach towns in the Hamptons. Queens is 108 square miles in size and is the largest of the five boroughs.

 
The Bronx: The Bronx is located directly north of Manhattan and is separated from that island via the Harlem River. It’s the northernmost of the five boroughs and is located on the mainland of New York state. Beyond its northern boundary are the small towns of suburban Westchester. The Bronx is connected to Manhattan with 12 bridges and tunnels and to Queens with three  spanning the East River. Its southernmost neighborhoods include Port Morris and Mott Haven, while Woodlawn lies along its northern edge. In the central Bronx are attractions like the New York Botanical Garden and the Bronx Zoo. The Bronx is around 42.5 square miles in size.

 
Staten Island: Like Manhattan, Staten Island is a separate landmass as well as a borough of New York City. This residential island does not offer bridge or railway access to or from Manhattan; in fact, the best way to get there from Manhattan is a free ride on the Staten Island Ferry. This journey across New York Harbor passes the small, iconic islands where the Statue of Liberty and the Ellis Island National Museum of are located. About 60 square miles in size, Staten Island is connected via bridges to New Jersey on the west and to Brooklyn on the east.

Basic NYC Geography: Avenues vs. Streets
Navigating gets a little dicey farther downtown, but everything above 14th Street is arranged by a precise grid system. Avenues go north-south, while streets go east to west. When you see a street called “West 33rd” or “East 102nd,” it designates whether it’s the side to the east or west of Fifth Avenue. Many neighborhood names, such as the East Village, are categorized this way too.

 
Basic NYC Geography: Major Manhattan Neighborhoods
Below are several Manhattan neighborhoods at a glance. Find out where each neighborhood begins and ends, as well as major features like sightseeing destinations, subway lines, and tips on how to navigate while you’re there. While not a complete list of neighborhoods, this is more than enough to get you started in the big city!

 
Midtown West


North/South Borders: West 59th Street (Central Park South) and West 34th Street
West/East Borders: The Hudson River and Fifth Avenue
Major train stations: Times Square, Penn Station, Columbus Circle
Major thoroughfares: Broadway, West 42nd Street, West 59th Street
Adjacent neighborhoods: Midtown East, Upper West Side, Chelsea
Navigation tip: Look for Times Square when you’re lost! It’s big and bright, and you can see it from a mile away. Use it to orient yourself and make sure you’re going in the right direction. (Same goes for the Empire State Building, also visible from miles around.)

 
grand central station
Photo by Brandon Nickerson on Unsplash
Midtown East

North/South Borders: East 59th Street (Central Park South) and East 34th Street
West/East Borders: Fifth Avenue and the East River
Train lines: 4, 5, 6
Major thoroughfares: East 42nd Street, Fifth Avenue, Park Avenue
Adjacent neighborhoods: Midtown West, Murray Hill, Upper East Side
Navigation tip: Starting from the East River, there are avenues numbered 1-3, followed by Lexington, Park, Madison, and Fifth Avenues, in order from east to west.

 
Upper West Side

North/South Borders: West 110th Street and West 59th Street (Central Park North and South)
West/East Borders: The Hudson River and Central Park West
Train lines: A, B, C, D, 1, 2, 3
Major thoroughfares: Broadway, Columbus Avenue, Amsterdam Avenue
Adjacent neighborhoods: West Harlem, Midtown West, Central Park
Navigation tip: Think of the Upper West Side as the area immediately west of the rectangular Central Park, which shares the same north/south borders.

 
Upper East Side

North/South Borders: East 96th Street and East 59th Street (Central Park South)
West/East Borders: Fifth Avenue and the East River
Train lines: 4, 5, 6, Q
Major thoroughfares: Lexington Avenue, Madison Avenue, Fifth Avenue
Adjacent neighborhoods: East Harlem, Midtown East, Central Park
Navigation tip: When you’re inside Central Park, look for black lampposts. The approximate cross-street is printed near the base with white paint to help you know how far north or south you’ve walked. For example, the inscription 6602 means you’re parallel with 66th Street, and this is the second lamppost to the west of Fifth Avenue.

 
Chelsea

North/South Borders: West 34th Street and West 14th Street
West/East Borders: Hudson River and Sixth Avenue (Avenue of the Americas)
Train lines: A, C, E, 1, 2, 3
Major thoroughfares: West 14th St., West 23rd St., Sixth Ave., Seventh Ave., Eighth Ave.
Adjacent neighborhoods: Midtown West, Greenwich Village, the West Village, Flatiron, NoMad
Navigation tip: The High Line park, a green space atop an old elevated rail line, runs parallel to 10th Avenue in Chelsea.

& nbsp;
washington square park pigeons
Photo by Caitlyn Wilson on Unsplash
Greenwich Village & the West Village

Known for: Washington Square Park
North/South Borders: West 14th Street and West Houston Street
West/East Borders: West Village (Hudson River and Sixth Ave.); Greenwich Village (Sixth Ave. and University Place)
Major train stations: West 4th Street, Union Square
Major thoroughfares: Bleecker Street, West 4th Street, Sixth Avenue, Seventh Avenue
Adjacent neighborhoods: SoHo, Chelsea, East Village
Navigation tip: The grid system falls apart in the West Village. If you see the intersection of West 10th and West 4th Streets, you’re not crazy! It’s just the slantwise streets that make the neighborhood unique.

 
SoHo

North/South Borders: West Houston Street and Canal Street
West/East Borders: Sixth Avenue and Crosby Street
Train lines: 1, A, C, E, Q, R, W
Major thoroughfares: Broadway, Prince Street, Spring Street
Adjacent neighborhoods: Greenwich Village/West Village, Little Italy, Nolita, Chinatown, Tribeca
Navigation tip: A twofer! SoHo means “SOuth of HOuston.” And here’s a mnemonic device for remembering downtown Manhattan’s street order, arranged from north to south: Prince spent the Spring Brooming the Grand Canal (with Howard Hester, the Mercer from Essex).

 
The Financial District

North/South Borders: Barclay Street and Battery Park
West/East Borders: Hudson River and the East River
Major train stations: Fulton Street, Brooklyn Bridge-City Hall
Major thoroughfares: Fulton Street, Broadway, Wall Street, Water Street
Adjacent neighborhoods: Tribeca, Chinatown, Lower East Side
Navigation tip: One of the most famous streets in the world, Broadway starts in the Financial District at Battery Park and extends the length of Manhattan and even into the Bronx! Broadway is nearly 34 miles long."""